# Multiprocessing zonal calculations

In [1]:
import sys, os, importlib, math, multiprocessing
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
import GOSTRocks.mapMisc as mapMisc
from GOSTRocks.misc import tPrint

sys.path.append("../src")
import h3_helper
import country_zonal

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
admin_bounds = "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM2.shp"
global_urban_file = "/home/public/Data/GLOBAL/GHSL/SMOD/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
global_pop_file = "/home/public/Data/GLOBAL/GHSL/Pop/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif"

In [10]:
inA = gpd.read_file(admin_bounds)
inA['ID'] = inA.index #Create ID for indexing

In [11]:
sel_iso3 = set(inA.loc[inA['WB_REGION'] == "LCR"]['WB_A3'].unique())
sel_iso3 = [x for x in sel_iso3 if x != None]

In [12]:
arg_list = []
for iso3 in sel_iso3:
    selA = inA.loc[inA['WB_A3'] == iso3].copy()
    out_folder = f"/home/wb411133/projects/Space2Stats/COUNTRIES/{iso3}"
    print(out_folder)
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    arg_list.append([iso3, selA, 6, global_pop_file, global_urban_file, out_folder])

/home/wb411133/projects/Space2Stats/COUNTRIES/ECU
/home/wb411133/projects/Space2Stats/COUNTRIES/PAN
/home/wb411133/projects/Space2Stats/COUNTRIES/COL
/home/wb411133/projects/Space2Stats/COUNTRIES/CHL
/home/wb411133/projects/Space2Stats/COUNTRIES/NIC
/home/wb411133/projects/Space2Stats/COUNTRIES/VCT
/home/wb411133/projects/Space2Stats/COUNTRIES/TTO
/home/wb411133/projects/Space2Stats/COUNTRIES/ARG
/home/wb411133/projects/Space2Stats/COUNTRIES/PER
/home/wb411133/projects/Space2Stats/COUNTRIES/HND
/home/wb411133/projects/Space2Stats/COUNTRIES/URY
/home/wb411133/projects/Space2Stats/COUNTRIES/PRY
/home/wb411133/projects/Space2Stats/COUNTRIES/SLV
/home/wb411133/projects/Space2Stats/COUNTRIES/GTM
/home/wb411133/projects/Space2Stats/COUNTRIES/GRD
/home/wb411133/projects/Space2Stats/COUNTRIES/BHS
/home/wb411133/projects/Space2Stats/COUNTRIES/VEN
/home/wb411133/projects/Space2Stats/COUNTRIES/BRB
/home/wb411133/projects/Space2Stats/COUNTRIES/CRI
/home/wb411133/projects/Space2Stats/COUNTRIES/BOL


In [13]:
def multi_processing_single_raster(sel_iso3, inA, h3_level, r_file, urban_file, out_folder, zonal_label):
    out_file = os.path.join(out_folder, f'{sel_iso3}_{zonal_label}_{h3_level}.csv')
    print(out_file)
    if not os.path.exists(out_file):
        zonalC = country_zonal.country_h3_zonal(sel_iso3, selA, "ID", h3_level, out_folder)
        zonal_res_pop = zonalC.zonal_raster_urban(r_file, urban_file)
        zonal_res_pop.to_csv(out_file)
    return(out_file)

In [14]:
multi_processing_single_raster(*arg_list[0], "GHS_POP_2020")

/home/wb411133/projects/Space2Stats/COUNTRIES/ECU/ECU_GHS_POP_2020_6.csv


Generating h3 grid level 6: 100%|██████████| 28/28 [00:00<00:00, 183.50it/s]


(1, 1827, 2113)


'/home/wb411133/projects/Space2Stats/COUNTRIES/ECU/ECU_GHS_POP_2020_6.csv'

In [ ]:
with multiprocessing.Pool(processes=min([70,len(arg_list)])) as pool:
    results = pool.starmap(multi_processing_single_raster, arg_list)

In [ ]:
multi_processing_single_raster(*arg_list[0], "GHS_POP_2020")